# Day 23: A Long Walk
The Elves resume water filtering operations! Clean water starts flowing over the edge of Island Island.

They offer to help you go over the edge of Island Island, too! Just hold on tight to one end of this impossibly long rope and they'll lower you down a safe distance from the massive waterfall you just created.

As you finally reach Snow Island, you see that the water isn't really reaching the ground: it's being absorbed by the air itself. It looks like you'll finally have a little downtime while the moisture builds up to snow-producing levels. Snow Island is pretty scenic, even without any snow; why not take a walk?

There's a map of nearby hiking trails (your puzzle input) that indicates paths (.), forest (#), and steep slopes (^, >, v, and <).

For example:
```
#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
````
You're currently on the single path tile in the top row; your goal is to reach the single path tile in the bottom row. Because of all the mist from the waterfall, the slopes are probably quite icy; if you step onto a slope tile, your next step must be downhill (in the direction the arrow is pointing). To make sure you have the most scenic hike possible, never step onto the same tile twice. What is the longest hike you can take?

In the example above, the longest hike you can take is marked with O, and your starting position is marked S:
```
#S#####################
#OOOOOOO#########...###
#######O#########.#.###
###OOOOO#OOO>.###.#.###
###O#####O#O#.###.#.###
###OOOOO#O#O#.....#...#
###v###O#O#O#########.#
###...#O#O#OOOOOOO#...#
#####.#O#O#######O#.###
#.....#O#O#OOOOOOO#...#
#.#####O#O#O#########v#
#.#...#OOO#OOO###OOOOO#
#.#.#v#######O###O###O#
#...#.>.#...>OOO#O###O#
#####v#.#.###v#O#O###O#
#.....#...#...#O#O#OOO#
#.#########.###O#O#O###
#...###...#...#OOO#O###
###.###.#.###v#####O###
#...#...#.#.>.>.#.>O###
#.###.###.#.###.#.#O###
#.....###...###...#OOO#
#####################O#
````
This hike contains 94 steps. (The other possible hikes you could have taken were 90, 86, 82, 82, and 74 steps long.)

Find the longest hike you can take through the hiking trails listed on your map. How many steps long is the longest hike?

In [34]:
from collections import defaultdict

def result(inputfile):

    input = []

    with open(inputfile) as f:
        for line in f:
            input.append(line.strip())
    toExplore = [[0,1]]
    toExploreNext = [[1,1]]

    length = len(input)
    width = len(input[0])

    def getNeighbours(pos):
        ns = []
        isPointingInto = []
        for i, shift in enumerate([[0,1], [0,-1], [1,0], [-1,0]]):
            ny = pos[0] + shift[0]
            nx = pos[1] + shift[1]
            if 0 <= ny < length and 0 <= nx < width:
                tile = input[ny][nx]
                if tile != '#':
                    ns.append([ny,nx])
                    if tile == '.':
                        isPointingInto.append('.')
                    else:
                        isPointingInto.append([i,tile] in [[0,'<'], [1,'>'], [2,'^'], [3,'v']])
        return ns, isPointingInto


    foundPathStarts = []
    foundPathEnds = []
    foundPathLengths = []
    while len(toExplore) > 0:
        s = toExplore.pop(0)
        start = s.copy()
        sNext = toExploreNext.pop(0)
        pathLength = 1
        while True:
            ns, isPointingInto = getNeighbours(sNext)
            if len(ns) > 2:
                deadEnd = False
                break
            elif len(ns) < 2:
                deadEnd = True
                break
            for n in ns:
                if n != s:
                    s = sNext
                    sNext = n
                    pathLength += 1
                    break
        if deadEnd:
            if sNext == [length - 1, width - 2]:
                foundPathStarts.append(start)
                foundPathEnds.append(sNext)
                foundPathLengths.append(pathLength)
        else:
            if sNext not in foundPathEnds:
                for i, n in enumerate(ns):
                    if n != s and isPointingInto[i] == False:
                        toExplore.append(sNext)
                        toExploreNext.append(n)
            foundPathStarts.append(start)
            foundPathEnds.append(sNext)
            foundPathLengths.append(pathLength)
    
    edgesPointingOut = defaultdict(lambda: [])
    edgesLengths = defaultdict(lambda: [])
    for start, end, l in zip(foundPathStarts, foundPathEnds, foundPathLengths):
        edgesPointingOut[str(start)].append(end)
        edgesLengths[str(start)].append(l)
    
    foundPaths = {}
    currentPath = [[0,1]]
    def goOutgoingPaths(start, currentLen):
        if start == [length - 1, width - 2]:
            foundPaths[str(currentPath)] = currentLen
        ends = edgesPointingOut[str(start)]
        ls = edgesLengths[str(start)]
        for end, l in zip(ends, ls):
            if end not in currentPath:
                currentPath.append(end)
                goOutgoingPaths(end, currentLen + l)
                currentPath.pop()
    goOutgoingPaths([0,1], 0)
    return max(foundPaths.values())

In [35]:
result("example-input.txt")

94

In [36]:
result("input.txt")

2086

### Part Two
As you reach the trailhead, you realize that the ground isn't as slippery as you expected; you'll have no problem climbing up the steep slopes.

Now, treat all slopes as if they were normal paths (.). You still want to make sure you have the most scenic hike possible, so continue to ensure that you never step onto the same tile twice. What is the longest hike you can take?

In the example above, this increases the longest hike to 154 steps:
```
#S#####################
#OOOOOOO#########OOO###
#######O#########O#O###
###OOOOO#.>OOO###O#O###
###O#####.#O#O###O#O###
###O>...#.#O#OOOOO#OOO#
###O###.#.#O#########O#
###OOO#.#.#OOOOOOO#OOO#
#####O#.#.#######O#O###
#OOOOO#.#.#OOOOOOO#OOO#
#O#####.#.#O#########O#
#O#OOO#...#OOO###...>O#
#O#O#O#######O###.###O#
#OOO#O>.#...>O>.#.###O#
#####O#.#.###O#.#.###O#
#OOOOO#...#OOO#.#.#OOO#
#O#########O###.#.#O###
#OOO###OOO#OOO#...#O###
###O###O#O###O#####O###
#OOO#OOO#O#OOO>.#.>O###
#O###O###O#O###.#.#O###
#OOOOO###OOO###...#OOO#
#####################O#
```
Find the longest hike you can take through the surprisingly dry hiking trails listed on your map. How many steps long is the longest hike?

In [46]:
def result2(inputfile):

    input = []

    with open(inputfile) as f:
        for line in f:
            input.append(line.strip())
    toExplore = [[0,1]]
    toExploreNext = [[1,1]]

    length = len(input)
    width = len(input[0])

    def getNeighbours(pos):
        ns = []
        isPointingInto = []
        for i, shift in enumerate([[0,1], [0,-1], [1,0], [-1,0]]):
            ny = pos[0] + shift[0]
            nx = pos[1] + shift[1]
            if 0 <= ny < length and 0 <= nx < width:
                tile = input[ny][nx]
                if tile != '#':
                    ns.append([ny,nx])
                    if tile == '.':
                        isPointingInto.append('.')
                    else:
                        isPointingInto.append([i,tile] in [[0,'<'], [1,'>'], [2,'^'], [3,'v']])
        return ns, isPointingInto


    foundPathStarts = []
    foundPathEnds = []
    foundPathLengths = []
    while len(toExplore) > 0:
        s = toExplore.pop(0)
        start = s.copy()
        sNext = toExploreNext.pop(0)
        pathLength = 1
        while True:
            ns, isPointingInto = getNeighbours(sNext)
            if len(ns) > 2:
                deadEnd = False
                break
            elif len(ns) < 2:
                deadEnd = True
                break
            for n in ns:
                if n != s:
                    s = sNext
                    sNext = n
                    pathLength += 1
                    break
        if deadEnd:
            if sNext == [length - 1, width - 2]:
                foundPathStarts.append(start)
                foundPathEnds.append(sNext)
                foundPathLengths.append(pathLength)
        else:
            if sNext not in foundPathEnds:
                for i, n in enumerate(ns):
                    if n != s and isPointingInto[i] == False:
                        toExplore.append(sNext)
                        toExploreNext.append(n)
            foundPathStarts.append(start)
            foundPathEnds.append(sNext)
            foundPathLengths.append(pathLength)
    
    edgesPointingOut = defaultdict(lambda: [])
    edgesLengths = defaultdict(lambda: [])
    for start, end, l in zip(foundPathStarts, foundPathEnds, foundPathLengths):
        edgesPointingOut[str(start)].append(end)
        edgesLengths[str(start)].append(l)
        edgesPointingOut[str(end)].append(start)
        edgesLengths[str(end)].append(l)
    
    foundPaths = {}
    currentPath = [[0,1]]
    def goOutgoingPaths(start, currentLen):
        if start == [length - 1, width - 2]:
            foundPaths[str(currentPath)] = currentLen
        ends = edgesPointingOut[str(start)]
        ls = edgesLengths[str(start)]
        for end, l in zip(ends, ls):
            if end not in currentPath:
                currentPath.append(end)
                goOutgoingPaths(end, currentLen + l)
                currentPath.pop()
    goOutgoingPaths([0,1], 0)
    return max(foundPaths.values())

In [47]:
result2("example-input.txt")

154

In [48]:
result2("input.txt")

6526